<h2>Section 1: Library Imports and Constants</h2>
<p>This section imports all the required Python libraries and sets up constants for colors, layout, and data types. These imports include widgets, plotting libraries, Firebase, date/time utilities, and more. The constants defined here are used throughout the notebook for consistent styling and data handling.</p>

In [6]:
!pip install firebase
!pip install -qU 'google-genai>=1.0.0' # Used for gemini chatbot.

import ipywidgets as widgets
from IPython.display import display, clear_output
import random
import time
import matplotlib.pyplot as plt
import io
from typing import Dict, List, Callable
from firebase import firebase
from datetime import datetime, timedelta
from zoneinfo import ZoneInfo  # Python 3.9+
import dateutil.parser         # Robust timestamp parsing
import matplotlib.dates as mdates
from nltk.stem import PorterStemmer

local_tz = ZoneInfo("Asia/Jerusalem")
# --- Constants ---
BLUE_COLOR = '#3498db'
DARKER_BLUE_COLOR = '#2c3e50'
ORANGE_COLOR = '#FF851B'
LIGHT_BLUE_COLOR = '#f0f8ff'
LIGHTER_BLUE_COLOR = '#f7f9fc'
WHITE_COLOR = 'rgba(255,255,255,0.7)'
LIGHT_GREY_BORDER = '#ddd'
BOX_SHADOW = '0px 4px 8px rgba(0,0,0,0.2)'
BORDER_RADIUS = '12px'
PADDING = '30px'
MARGIN = '20px'

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.5/199.5 kB 5.6 MB/s eta 0:00:00


<h2>Section 2: Data Types, Firebase Setup, and Dummy Data</h2>
<p>This section defines the data types used for sensor data and engineers' performance, sets up the Firebase connection for real-time database access, and provides example (dummy) data for daily tasks and engineers. These elements are foundational for the dashboard's data-driven features.</p>

In [7]:
# --- Data Types ---
SensorData = Dict[str, Dict[str, float or str]]
EngineersPerformance = Dict[str, Dict[str, int]]
HistoricalSensorData = Dict[str, Dict[str, List[float] or List[str]]]

stemmer = PorterStemmer()

# --- Firebase Setup ---
FBconn = firebase.FirebaseApplication(
    'https://cloudteamwolf-default-rtdb.europe-west1.firebasedatabase.app', None
)
local_tz = ZoneInfo("Asia/Jerusalem")
DATABASE_PATH = 'Data'
INDOOR_SENSOR_PATH = f'{DATABASE_PATH}/indoor'
OUTDOOR_SENSOR_PATH = f'{DATABASE_PATH}/outdoor'

# --- Dummy Data (Replace with your actual data) ---
daily_tasks: List[str] = [
    "Review production logs for anomalies",
    "Check calibration of robotic arms",
    "Optimize energy consumption routines",
    "Inspect conveyor belt system",
]

engineers_performance: EngineersPerformance = {
    "Alice": {"Points": 1550, "Improvements Completed": 32},
    "Bob": {"Points": 1200, "Improvements Completed": 25},
    "Charlie": {"Points": 1800, "Improvements Completed": 40},
    "David": {"Points": 950, "Improvements Completed": 18},
}

<h2>Section 3: Helper Functions for Widgets and Search</h2>
<p>This section provides reusable helper functions for creating styled widgets (buttons, text inputs, dropdowns, layouts) and for performing search queries. These functions help keep the code modular and make it easy to build interactive UI components for the dashboard.</p>

In [8]:
# --- Helper Functions (Refactored for Clarity) ---
def create_button(description: str, button_style: str = 'primary', **layout_kwargs) -> widgets.Button:
    layout = widgets.Layout(
        width=layout_kwargs.get('width', '200px'),
        height=layout_kwargs.get('height', '60px'),
        border_radius=layout_kwargs.get('border_radius', BORDER_RADIUS),
        font_size=layout_kwargs.get('font_size', '16px'),
        margin=layout_kwargs.get('margin', '10px'),
        box_shadow=layout_kwargs.get('box_shadow', None)
    )
    return widgets.Button(description=description, button_style=button_style, layout=layout)

def create_text_input(description: str, **layout_kwargs) -> widgets.Text:
    layout = widgets.Layout(
        width=layout_kwargs.get('width', '600px'),
        height=layout_kwargs.get('height', '40px'),
        border_radius=layout_kwargs.get('border_radius', BORDER_RADIUS),
        padding=layout_kwargs.get('padding', '10px'),
        font_size=layout_kwargs.get('font_size', '16px')
    )
    return widgets.Text(description=description, layout=layout)

def create_output(**layout_kwargs) -> widgets.Output:
    layout = widgets.Layout(**layout_kwargs)
    return widgets.Output(layout=layout)

def create_dropdown(options: list, description: str, **layout_kwargs) -> widgets.Dropdown:
    layout = widgets.Layout(
        width=layout_kwargs.get('width', '300px'),
        margin=layout_kwargs.get('margin', '10px 0')
    )
    return widgets.Dropdown(options=options, description=description, layout=layout)

def create_vbox(children: list, **layout_kwargs) -> widgets.VBox:
    layout = widgets.Layout(
        padding=layout_kwargs.get('padding', PADDING),
        border=layout_kwargs.get('border', f'2px solid {BLUE_COLOR}'),
        border_radius=layout_kwargs.get('border_radius', '15px'),
        margin=layout_kwargs.get('margin', MARGIN),
        background_color=layout_kwargs.get('background_color', LIGHT_BLUE_COLOR)
    )
    return widgets.VBox(children=children, layout=layout)

def create_hbox(children: list, **layout_kwargs) -> widgets.HBox:
    layout = widgets.Layout(**layout_kwargs)
    return widgets.HBox(children=children, layout=layout)

# --- Use HtmlWidgetFactory for all HTML widgets ---
# (No change needed for create_html_widget, already mapped)

# --- Clean up perform_search for clarity ---
def perform_search(query: str):
    stemmed_query = stemmer.stem(query.lower())
    index_data = FBconn.get('/Data/Index', None)
    results = []
    if index_data and stemmed_query in index_data:
        entry = index_data[stemmed_query]
        results.append(f"<h3>Search Result for: <i>{query}</i></h3>")
        results.append(f"<b>Term:</b> {entry['term']}")
        results.append(f"<b>Count:</b> {entry['count']} occurrence(s)")
        results.append("<b>Appears in URLs:</b><ul>")
        # DocIDs is a list of dicts
        results.extend([
            f"<li><a href='{item['url']}' target='_blank'>{item['url']}</a> <span style='color:black;'>({item['count']} times)</span></li>"
            for item in entry.get('DocIDs', [])
        ])
        results.append("</ul>")
    else:
        results.append(f"<b>No matching word</b> found in the top 100 index for: <i>{query}</i>")
    return results

<h2>Section 4: Sensor Data Fetching and Statistics</h2>
<p>This section contains functions to fetch the latest and historical sensor data from Firebase, generate sensor statistics, and create plots for data visualization. These functions are essential for displaying real-time and historical sensor information in the dashboard.</p>

In [9]:
# --- Clean up generate_sensor_statistics for clarity ---
def generate_sensor_statistics(selected_sensor: str, sensor_data: SensorData, selected_graph_attribute: str = None) -> widgets.HTML:
    data = sensor_data.get(selected_sensor, {})
    if not data:
        return create_html_widget(f"No data available for {selected_sensor}.")
    found_key = next((k for k in data if selected_graph_attribute and k.lower() == selected_graph_attribute.lower()), None)
    if found_key:
        units_map = {
            "Indoor Sensor": {"Humidity": "%", "Temperature": "°C", "Pressure": "Bar", "Distance": "Cm"},
            "Outdoor Sensor": {"Humidity": "%", "Temperature": "°C", "DLight": "Lux"}
        }
        unit = units_map.get(selected_sensor, {}).get(selected_graph_attribute, "")
        stats = f"<b>{selected_graph_attribute}:</b> {data[found_key]} {unit}<br>"
        if "Time" in data:
            stats += f"<b>Time:</b> {data['Time']}<br>"
    else:
        stats = f"<b>No specific data available for '{selected_graph_attribute}' or attribute not found in latest sensor data.</b><br>"
    return create_html_widget(f"<div style='display: flex; justify-content: space-between;'><div style='flex: 1; margin-right: 20px;'>{stats}</div></div>")

def fetch_latest_sensor_data() -> dict:
    """Fetch the latest value for each attribute from indoor and outdoor sensor data."""

    def get_latest_all_fields(data):
        if not data:
            return {}

        # Flatten all keys to get the full set of attributes ever recorded
        all_possible_keys = set()
        for record in data.values():
            all_possible_keys.update(record.keys())

        # Now scan in reverse time order and collect the latest value for each attribute
        latest_values = {}
        sorted_items = sorted(data.items(), key=lambda x: int(x[0]), reverse=True)

        for _, record in sorted_items:
            for key in all_possible_keys:
                if key not in latest_values and key in record:
                    latest_values[key] = record[key]
            if len(latest_values) == len(all_possible_keys):
                break

        return latest_values

    indoor_data = FBconn.get(INDOOR_SENSOR_PATH, None)
    outdoor_data = FBconn.get(OUTDOOR_SENSOR_PATH, None)

    indoor_latest = get_latest_all_fields(indoor_data)
    outdoor_latest = get_latest_all_fields(outdoor_data)

    return {
        "Indoor Sensor": indoor_latest,
        "Outdoor Sensor": outdoor_latest,
    }

def fetch_historical_sensor_data(sensor_path: str) -> HistoricalSensorData:
    """
    Fetches all historical sensor data for a given sensor path.
    The data is sorted by timestamp and includes all available attributes.
    """
    data = FBconn.get(sensor_path, None)
    israel_tz = ZoneInfo("Asia/Jerusalem")

    historical_data: HistoricalSensorData = {"Timestamp": [], "Humidity": [], "Temperature": []}
    if sensor_path == INDOOR_SENSOR_PATH:
        historical_data.update({"Pressure": [], "Distance": []})
    elif sensor_path == OUTDOOR_SENSOR_PATH:
        historical_data.update({"DLight": []})

    if not data:
        return historical_data

    # Sort data by timestamp to ensure correct plotting order
    sorted_entries = sorted(data.values(), key=lambda x: dateutil.parser.parse(f"{x.get('Date', '')} {x.get('Time', '')}") if x.get('Date') and x.get('Time') else datetime.min)

    for entry in sorted_entries: # Iterate through sorted entries
        date_str = entry.get("Date", "")
        time_str = entry.get("Time", "")
        full_timestamp_str = f"{date_str} {time_str}" if date_str and time_str else ""
        try:
            entry_time = dateutil.parser.parse(full_timestamp_str)
            if entry_time.tzinfo is None:
                entry_time = entry_time.replace(tzinfo=ZoneInfo("UTC")).astimezone(israel_tz)
            else:
                entry_time = entry_time.astimezone(israel_tz)
        except Exception:
            # Skip entries with invalid time formats
            continue

        # No time limit check here, append all valid entries
        historical_data["Timestamp"].append(entry_time)
        historical_data["Humidity"].append(entry.get("Humidity"))
        historical_data["Temperature"].append(entry.get("Temperature"))
        if sensor_path == INDOOR_SENSOR_PATH:
            historical_data["Pressure"].append(entry.get("Pressure"))
            historical_data["Distance"].append(entry.get("Distance"))
        elif sensor_path == OUTDOOR_SENSOR_PATH:
            historical_data["DLight"].append(entry.get("DLight"))
    return historical_data


def create_historical_plot(sensor_name: str, selected_attribute: str) -> widgets.Output:
    """Creates and returns an output widget with a single historical plot for the selected attribute.
    Creates a fluid connected line even when data points are missing."""

    # Centralized configuration for sensors, units, and attribute-specific colors
    sensor_configurations = {
        "Indoor Sensor": {
            "path": INDOOR_SENSOR_PATH,
            "units": {
                "Humidity": "%",
                "Temperature": "°C",
                "Pressure": "Bar",
                "Distance": "Cm"
            },
            "colors": { # Define specific colors for each attribute of the Indoor Sensor
                "Humidity": 'tab:blue',
                "Temperature": 'tab:red',
                "Pressure": 'tab:green',
                "Distance": 'tab:purple'
            }
        },
        "Outdoor Sensor": {
            "path": OUTDOOR_SENSOR_PATH,
            "units": {
                "Humidity": "%",
                "Temperature": "°C",
                "DLight": "Lux"
            },
            "colors": { # Define specific colors for each attribute of the Outdoor Sensor
                "Humidity": 'tab:orange',
                "Temperature": 'tab:cyan',
                "DLight": 'tab:brown'
            }
        }
    }

    # Get the configuration for the selected sensor
    config = sensor_configurations.get(sensor_name)

    # Handle invalid sensor name
    if not config:
        error_output = widgets.Output(layout=widgets.Layout(height='auto'))
        with error_output:
            display(widgets.HTML(f"<b>❌ Invalid sensor name:</b> {sensor_name}"))
        return error_output

    # Fetch historical data using the path from the configuration
    historical_data = fetch_historical_sensor_data(config["path"])
    units = config["units"]
    attribute_colors = config["colors"] # Get the specific colors for this sensor

    # Determine the color for the selected attribute
    color = attribute_colors.get(selected_attribute, 'gray') # Default to gray if no specific color is found

    # Handle case where no historical data is available
    if not historical_data or not historical_data["Timestamp"]:
        no_data_output = widgets.Output(layout=widgets.Layout(height='auto'))
        with no_data_output:
            display(widgets.HTML(f"<b>📭 No historical data available for {sensor_name}</b>"))
        return no_data_output

    out = widgets.Output()
    with out:
        plt.figure(figsize=(10, 6))

        times = historical_data["Timestamp"]
        y_data = historical_data.get(selected_attribute, [])
        unit = units.get(selected_attribute, "")

        if not y_data:
            # Display message if the attribute exists but has no data
            plt.text(0.5, 0.5, f"No data available for '{selected_attribute}'", ha='center', va='center', fontsize=12)
            plt.title(f"{selected_attribute} over Time ({sensor_name})", fontweight='bold')
        else:
            # Filter out None values to create a fluid connected line
            valid_points = [(t, y) for t, y in zip(times, y_data) if y is not None]

            if valid_points:
                valid_times, valid_y_data = zip(*valid_points)

                # Plot main line
                plt.plot(valid_times, valid_y_data, marker='o', color=color,
                                 linewidth=2, markersize=5, label=selected_attribute)

                # Shadow line (no label) for visual thickness
                plt.plot(valid_times, valid_y_data, color=color, alpha=0.3, linewidth=4)

                # Axis formatting and labels
                plt.title(f"{selected_attribute} over Time ({sensor_name})", fontweight='bold')
                plt.xlabel("Time", fontweight='bold')
                plt.ylabel(f"{selected_attribute} ({unit})", fontweight='bold')
                plt.grid(True, linestyle='--', alpha=0.4)

                # Format x-axis for datetime objects
                if valid_times and isinstance(valid_times[0], datetime): # Check if times are datetime objects
                    ax = plt.gca()
                    ax.xaxis.set_major_formatter(mdates.DateFormatter('%H:%M:%S'))
                    ax.xaxis.set_major_locator(mdates.AutoDateLocator())
                    plt.setp(ax.get_xticklabels(), rotation=45, ha='right')

                # Add y-axis padding for better visualization
                y_min, y_max = min(valid_y_data), max(valid_y_data)
                if y_min != y_max: # Only apply padding if there's a range
                    y_range = y_max - y_min
                    plt.ylim(y_min - 0.15 * y_range, y_max + 0.15 * y_range)
                else: # Handle case where all y_data points are the same
                    plt.ylim(y_min - 1, y_max + 1) # Small fixed padding

                plt.legend()
            else:
                # This case handles when y_data contains only None values
                plt.text(0.5, 0.5, f"No valid data points for '{selected_attribute}'",
                                 ha='center', va='center', fontsize=12)
                plt.title(f"{selected_attribute} over Time ({sensor_name})", fontweight='bold')

        plt.tight_layout() # Adjust plot to prevent labels from overlapping
        plt.show()
    return out

def update_sensor_data_and_plot(change) -> None:
    """Updates the statistics display and plots based on the selected sensor and attribute."""
    selected_sensor = sensor_selector.value # Get the current sensor selected
    selected_attribute = graph_selector.value # Get the current attribute selected

    latest_data = fetch_latest_sensor_data()
    # Update the display for selected sensor and graph
    # Pass selected_attribute to generate_sensor_statistics
    stats_display.value = generate_sensor_statistics(selected_sensor, latest_data, selected_attribute).value

    # Clear the output widget and display the new figure
    stats_plots.clear_output(wait=True)
    with stats_plots:
        out = create_historical_plot(selected_sensor, selected_attribute)
        display(out)

<h2>Section 5: HTML Widget Factory and Main Menu Widgets</h2>
<p>This section introduces the HtmlWidgetFactory class for creating standardized HTML widgets and sets up the main menu widgets, including the dashboard summary and navigation buttons. These elements form the visual and interactive entry point of the dashboard.</p>

In [10]:
# --- HTML Widget Factory ---
class HtmlWidgetFactory:
    """Factory for creating standardized HTML widgets for the dashboard."""
    @staticmethod
    def title(text, level=1, color=BLUE_COLOR, align='center', margin='0 0 20px 0'):
        return widgets.HTML(f"<h{level} style='color:{color}; text-align:{align}; margin-bottom:30px;'>{text}</h{level}>", layout=widgets.Layout(margin=margin))

    @staticmethod
    def status_box(title, content, icon=None, color=LIGHTER_BLUE_COLOR):
        icon_html = f"<span style='font-size:1.5em; margin-right:10px;'>{icon}</span>" if icon else ""
        return widgets.HTML(f"""
            <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{color}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                    {icon_html}{title}
                </h4>
                {content}
            </div>
        """)

    @staticmethod
    def alert(message, color='#ffeaea', text_color='#b71c1c'):
        return widgets.HTML(f"<div style='color:{text_color}; font-weight:bold; font-size:1.1em; background-color:{color}; border-radius:8px; padding:10px; margin-bottom:10px; box-shadow:{BOX_SHADOW};'>{message}</div>")

    @staticmethod
    def html_list(items, ordered=False):
        tag = 'ol' if ordered else 'ul'
        return widgets.HTML(f"<{tag} style='padding-left:20px;'>" + ''.join([f"<li>{item}</li>" for item in items]) + f"</{tag}>")

    @staticmethod
    def info_box(content, background=LIGHTER_BLUE_COLOR):
        return widgets.HTML(f"<div style='padding:15px; background-color:{background}; border-radius:10px; box-shadow:{BOX_SHADOW}; border: 1px solid {BLUE_COLOR};'>{content}</div>")

    @staticmethod
    def simple_html(value, **layout_kwargs):
        layout = widgets.Layout(**layout_kwargs)
        return widgets.HTML(value=value, layout=layout)

# Replace create_html_widget with HtmlWidgetFactory.simple_html
create_html_widget = HtmlWidgetFactory.simple_html

# --- Main Menu Screen Widgets ---
main_menu_title = create_html_widget(
    "<h1 style='color:#3498db; text-align:center; margin-bottom:30px;'>Robotics Lab Control Center</h1>",
    margin='0 0 20px 0'
)

# Create large, attractive menu buttons
manager_btn = create_button(
    "System Manager",
    button_style='info',
    width='280px',
    height='80px',
    margin='10px',
    font_weight='bold',
    box_shadow=BOX_SHADOW
)

statistics_btn = create_button(
    "Sensor Statistics",
    button_style='success',
    width='280px',
    height='80px',
    margin='10px',
    font_weight='bold',
    box_shadow=BOX_SHADOW
)

search_btn = create_button(
    "Search Query",
    button_style='warning',
    width='280px',
    height='80px',
    margin='10px',
    font_weight='bold',
    box_shadow=BOX_SHADOW
)

optimization_btn = create_button(
    "Optimization Race",
    button_style='danger',
    width='280px',
    height='80px',
    margin='10px',
    font_weight='bold',
    box_shadow=BOX_SHADOW
)

# Fetch the latest sensor data right before creating the dashboard_summary
# This ensures the status is up-to-date when the dashboard_summary is rendered.
latest_sensor_info = fetch_latest_sensor_data()
indoor_sensor_status_dynamic = "Available" if latest_sensor_info.get("Indoor Sensor") else "Not Available"
outdoor_sensor_status_dynamic = "Available" if latest_sensor_info.get("Outdoor Sensor") else "Not Available"

# Create dashboard summary for main menu
dashboard_summary = create_html_widget(
    f"""
    <div style='padding:15px; background-color:white; border-radius:10px; box-shadow:{BOX_SHADOW}; border: 1px solid {BLUE_COLOR};'>
        <h3 style='color:{DARKER_BLUE_COLOR}; text-align:center; margin-bottom:15px; font-size: 1.5em;'>Dashboard Summary</h3>
        <div style='display:flex; justify-content:space-around; flex-wrap:wrap; gap: 15px;'>
            <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{LIGHTER_BLUE_COLOR}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                    <span style='font-size:1.5em; margin-right:10px;'>📡</span>Sensors
                </h4>
                <p style='margin-bottom:5px; font-size:1.1em;'>
                    <span style='font-weight:bold;'>Indoor:</span> {indoor_sensor_status_dynamic}
                </p>
                <p style='margin-bottom:0; font-size:1.1em;'>
                    <span style='font-weight:bold;'>Outdoor:</span> {outdoor_sensor_status_dynamic}
                </p>
            </div>
            <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{LIGHTER_BLUE_COLOR}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                    <span style='font-size:1.5em; margin-right:10px;'>📋</span>Daily Tasks
                </h4>
                <p style='margin-bottom:0; font-size:1.1em;'><span style='font-weight:bold;'>{len(daily_tasks)}</span> tasks pending</p>
            </div>
            <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{LIGHTER_BLUE_COLOR}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                    <span style='font-size:1.5em; margin-right:10px;'>🏆</span>Optimization Race
                </h4>
                <p style='margin-bottom:0; font-size:1.1em;'>Leader: <span style='font-weight:bold;'>Charlie</span> with <span style='color:{ORANGE_COLOR}; font-weight:bold;'>1800</span> points</p>
            </div>
        </div>
    </div>
    """,
    margin='20px 0'
)

# Arrange menu buttons in a grid layout
top_row = create_hbox([manager_btn, statistics_btn], justify_content='center')
bottom_row = create_hbox([search_btn, optimization_btn], justify_content='center')
menu_buttons = create_vbox(
    [top_row, bottom_row],
    border='none',
    background_color='transparent',
    padding='10px'
)

# Welcome message with date/time
current_time = datetime.now(local_tz).strftime("%A, %B %d, %Y %I:%M %p")
welcome_msg = create_html_widget(
    f"<div style='text-align:center; margin:10px 0 30px 0;'><h3>Welcome to Robotics Lab Control Center</h3><p>{current_time}</p></div>"
)

# Main menu container
main_menu_container = create_vbox(
    [
        main_menu_title,
        welcome_msg,
        dashboard_summary,
        menu_buttons,
        gemini_chatbot_widget()
    ],
    background_color=LIGHTER_BLUE_COLOR,
    border=f'2px solid {DARKER_BLUE_COLOR}',
    box_shadow=BOX_SHADOW,
    width='800px',
    margin='0 auto'
)

NameError: name 'gemini_chatbot_widget' is not defined

<h2>Section 6: System Manager Screen Widgets</h2>
<p>This section defines the widgets and layout for the System Manager screen, including sensor status, daily tasks, and the engineer leaderboard. It also includes functions to update sensor status and display alerts for anomalies, providing a comprehensive management interface.</p>

In [ ]:
# --- System Manager Screen Widgets ---
manager_title = create_html_widget("<h2 style='color:#3498db;'>Robotics Lab System Manager Dashboard</h2>")
production_status_title = create_html_widget("<h3>Sensor Status</h3>")
sensor_status_display = create_html_widget("") # To display indoor/outdoor status
tasks_title = create_html_widget("<h3>Daily Optimization Tasks</h3>")
tasks_list = create_html_widget("<ul style='list-style-type:disc; padding-left:20px;'>" + "".join([f"<li>{task}</li>" for task in daily_tasks]) + "</ul>")
leaderboard_title = create_html_widget("<h3>Engineer Optimization Race Leaderboard</h3>")
leaderboard_items = [f"<li><span style='font-weight:bold;'>{engineer}</span>: <span style='color:{ORANGE_COLOR};'>{performance['Points']}</span> Points, <span style='color:{ORANGE_COLOR};'>{performance['Improvements Completed']}</span> Improvements</span></li>"
                     for engineer, performance in engineers_performance.items()]
leaderboard = create_html_widget("<ol style='padding-left:20px;'>" + "".join(leaderboard_items) + "</ol>")

manager_back_btn = create_button("Back to Main Menu", button_style='info')
manager_output = create_output()
manager_container = create_vbox(
    [
        manager_title,
        production_status_title,
        widgets.VBox([sensor_status_display], layout=widgets.Layout(border=f'1px solid {LIGHT_GREY_BORDER}', border_radius='8px', padding='15px', margin_bottom='15px',
                                                                     background_color=WHITE_COLOR)),
        tasks_title,
        tasks_list,
        leaderboard_title,
        leaderboard,
        manager_back_btn,
        manager_output,
        gemini_chatbot_widget()
    ]
)

def update_sensor_status() -> None:
    """Updates the sensor status display on the manager screen."""
    latest_data = fetch_latest_sensor_data()
    indoor_status = "Available" if latest_data.get("Indoor Sensor") else "Not Available"
    outdoor_status = "Available" if latest_data.get("Outdoor Sensor") else "Not Available"
    status_text = f"""
        <b>Indoor Sensor:</b> <span style='font-weight:bold;'>{indoor_status}</span><br>
        <b>Outdoor Sensor:</b> <span style='font-weight:bold;'>{outdoor_status}</span>
    """
    sensor_status_display.value = status_text

# --- Anomaly Detection Helper (Reimplemented) ---
def detect_sensor_anomalies(sensor_data: dict) -> list:
    """Returns a list of anomaly messages if any sensor value is out of normal range."""
    anomalies = []
    # Define normal ranges for each attribute
    normal_ranges = {
        "Temperature": (16, 24),   # Celsius
        "Humidity": (30, 50),     # Percent
        "Pressure": (0.9, 1.1),   # Bar
        "Distance": (0, 100),     # Cm
        "DLight": (600,800),    # Lux
    }
    for sensor_name, data in sensor_data.items():
        if not data:
            continue
        for key, (min_v, max_v) in normal_ranges.items():
            value = data.get(key)
            if value is not None:
                try:
                    val = float(value)
                    if val < min_v or val > max_v:
                        anomalies.append(
                            f"<span style='font-weight:bold; color:#fff; background:linear-gradient(90deg,#ff1744,#ff9100); padding:2px 8px; border-radius:6px;'>{sensor_name}:</span> "
                            f"<span style='color:#ff1744; font-weight:bold;'>{key} = {val}</span> "
                            f"<span style='color:#fff; background:#333; border-radius:4px; padding:1px 6px; font-size:0.95em;'>(Normal: {min_v}-{max_v})</span>"
                        )
                except Exception:
                    continue
    return anomalies

# --- Manager Alert Widget (Simple Style, Original) ---
manager_alert = create_html_widget(
    "",
    border_radius='8px',
    padding='10px',
    margin_bottom='10px',
    background_color='#ffeaea',
    box_shadow=BOX_SHADOW
)

def update_manager_alert() -> None:
    """Updates the anomaly alert display on the manager screen."""
    latest_data = fetch_latest_sensor_data()
    anomalies = detect_sensor_anomalies(latest_data)
    if anomalies:
        alert_html = (
            "<div style='color:#b71c1c; font-weight:bold; font-size:1.1em;'>"
            "<span style='font-size:1.5em; vertical-align:middle; margin-right:10px;'>⚡</span>"
            "Sensor Anomaly Detected!<br>"
            + "<br>".join(anomalies) +
            "</div>"
        )
        manager_alert.value = alert_html
    else:
        manager_alert.value = ""  # No alert

# --- Add alert widget to manager_container (at the top) ---
manager_container.children = (manager_alert,) + manager_container.children

def show_manager_screen() -> None:
    """Displays the manager screen."""
    global current_screen, displayed_screen
    current_screen = "manager"
    clear_output(wait=True)
    displayed_screen.children = [manager_container]
    display(displayed_screen)
    update_sensor_status()
    update_manager_alert()  # <-- Add this line

<h2>Section 7: Search Query and Statistics Screen Widgets</h2>
<p>This section sets up the widgets and layouts for the Search Query and Sensor Statistics screens. It includes input fields, buttons, output areas, and dropdowns for selecting sensors and attributes, enabling users to interactively explore and analyze sensor data.</p>

In [ ]:
# --- Search Query Screen Widgets ---
query_title = create_html_widget("<h2 style='color:#3498db;'>Search Query</h2>")
query_input = widgets.Text(
    description="Enter your query:",
    style={'description_width': 'initial'},
    layout=widgets.Layout(
        width='600px',
        height='40px',
        border_radius=BORDER_RADIUS,
        padding='10px',
        font_size='16px'
    )
)
search_button = create_button("Search")
query_results = create_output(layout=widgets.Layout(height='300px', border=f'1px solid {LIGHT_GREY_BORDER}', border_radius='8px', overflow='auto', background_color='rgba(255,255,255,0.8)', padding='15px'))
query_back_btn = create_button("Back to Main Menu", button_style='info')
query_container = create_vbox(
    [
        query_title,
        query_input,
        search_button,
        query_results,
        query_back_btn,
        gemini_chatbot_widget()
    ]
)

# --- Statistics Screen Widgets ---
stats_title = create_html_widget("<h2 style='color:#2c3e50; text-align:center;'>Sensor Statistics</h2>")
stats_display = create_html_widget(
    "",
    border_radius='8px',
    padding='15px',
    margin_bottom='15px',
    background_color='rgba(255,255,255,0.9)',
    box_shadow=BOX_SHADOW
)


sensor_selector = create_dropdown(
    options=["Indoor Sensor", "Outdoor Sensor"],
    description="<b style='color:#2c3e50;'>Sel_Sensor:</b>",
    style={'description_width': '150px', 'description_justify': 'flex-start'}, # Added this line
    layout=widgets.Layout(width='auto', min_width='350px', margin='10px 0')
)

graph_selector = create_dropdown(
    options=["Humidity", "Temperature", "Pressure", "Distance"], # Default options, will be updated
    description="<b style='color:#2c3e50;'>Sel_Graph:</b>",
    style={'description_width': '150px', 'description_justify': 'flex-start'},
    layout=widgets.Layout(width='auto', min_width='350px', margin='10px 0')
)



selected_info_display = create_html_widget(
    "",
    border_radius='8px',
    padding='10px',
    margin_bottom='10px',
    background_color='rgba(240,248,255,0.8)',
    box_shadow=BOX_SHADOW
)

stats_back_btn = create_button("Back to Main Menu", button_style='info', layout=widgets.Layout(width='200px', height='50px', border_radius='10px', font_size='16px', margin='20px auto'))

# Create a dedicated output widget for plots with proper layout
stats_plots = widgets.Output(layout=widgets.Layout(
    min_height='600px',
    width='100%',
    border=f'1px solid {LIGHT_GREY_BORDER}',
    border_radius='8px',
    background_color='rgba(240,248,255,0.8)',
    box_shadow=BOX_SHADOW,
    padding='20px',
    overflow='visible'
))

stats_container = create_vbox(
    [
        stats_title,
        sensor_selector,
        graph_selector,
        selected_info_display,
        stats_display,
        stats_plots,
        stats_back_btn,
        gemini_chatbot_widget()
    ],
    layout=widgets.Layout(padding='30px', border=f'2px solid {DARKER_BLUE_COLOR}', border_radius='15px', margin='20px auto', background_color=LIGHTER_BLUE_COLOR, box_shadow=BOX_SHADOW)
)

<h2>Section 8: Optimization Race Screen Widgets</h2>
<p>This section defines the widgets and layout for the Optimization Race screen, including the leaderboard and daily tasks for engineers. It also provides a function to update the leaderboard, fostering a sense of competition and engagement among users.</p>

In [ ]:
# --- Optimization Race Screen Widgets ---
optimization_title = create_html_widget("<h2 style='color:#3498db;'>Optimization Race</h2>")
optimization_leaderboard_title = create_html_widget("<h3>Current Standings</h3>")
optimization_leaderboard = create_html_widget("")
optimization_tasks_title = create_html_widget("<h3>Your Daily Tasks</h3>")
optimization_tasks_list = create_html_widget("<ul style='list-style-type:disc; padding-left:20px;'>" + "".join([f"<li>{task}</li>" for task in daily_tasks]) + "</ul>")
optimization_back_btn = create_button("Back to Main Menu", button_style='info')
optimization_output = create_output()
optimization_container = create_vbox(
    [
        optimization_title,
        optimization_leaderboard_title,
        optimization_leaderboard,
        optimization_tasks_title,
        optimization_tasks_list,
        optimization_back_btn,
        optimization_output
    ]
)

def update_optimization_leaderboard() -> None:
    """Updates the leaderboard displayed on the optimization screen."""
    sorted_engineers = sorted(engineers_performance.items(), key=lambda item: item[1]['Points'], reverse=True)
    leaderboard_items = []
    medal_icons = {
        0: "🥇",
        1: "🥈",
        2: "🥉"
    }
    for i, (engineer, performance) in enumerate(sorted_engineers):
        medal = medal_icons.get(i, "")
        leaderboard_items.append(f"<li>{medal} <span style='font-weight:bold;'>{engineer}</span>: <span style='color:{ORANGE_COLOR};'>{performance['Points']}</span> Points, <span style='color:{ORANGE_COLOR};'>{performance['Improvements Completed']}</span> Improvements</span></li>")
    optimization_leaderboard.value = "<ol style='padding-left:20px;'>" + "".join(leaderboard_items) + "</ol>"

<h2>Section 9: Screen Management Functions</h2>
<p>This section contains functions to manage the display and navigation between different screens in the dashboard, such as the main menu, manager, query, statistics, and optimization screens. These functions ensure a smooth and dynamic user experience.</p>

In [ ]:
#--- ScreenManagement Functions ---
current_screen = "main_menu"
displayed_screen = widgets.VBox([])

def show_main_menu() -> None:
    """Displays the main menu screen and updates the current time."""
    global current_screen, displayed_screen
    current_screen = "main_menu"
    clear_output(wait=True)

    # Update the welcome message with the current time right before displaying
    current_time_str = datetime.now(local_tz).strftime("%A, %B %d, %Y %I:%M %p")
    welcome_msg.value = f"<div style='text-align:center; margin:10px 0 30px 0;'><h3>Welcome to Robotics Lab Control Center</h3><p>{current_time_str}</p></div>"

    # Re-fetch and update sensor status for the dashboard summary when returning to main menu
    global dashboard_summary # Ensure dashboard_summary is accessible
    latest_sensor_info = fetch_latest_sensor_data()
    indoor_sensor_status_dynamic = "Available" if latest_sensor_info.get("Indoor Sensor") else "Not Available"
    outdoor_sensor_status_dynamic = "Available" if latest_sensor_info.get("Outdoor Sensor") else "Not Available"

    # Re-create dashboard_summary to reflect the new sensor status
    # This might be less efficient if dashboard_summary is very complex,
    # but for HTML widgets, it's generally fine.
    dashboard_summary.value = f"""
        <div style='padding:15px; background-color:white; border-radius:10px; box-shadow:{BOX_SHADOW}; border: 1px solid {BLUE_COLOR};'>
            <h3 style='color:{DARKER_BLUE_COLOR}; text-align:center; margin-bottom:15px; font-size: 1.5em;'>Dashboard Summary</h3>
            <div style='display:flex; justify-content:space-around; flex-wrap:wrap; gap: 15px;'>
                <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{LIGHTER_BLUE_COLOR}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                    <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                        <span style='font-size:1.5em; margin-right:10px;'>📡</span>Sensors
                    </h4>
                    <p style='margin-bottom:5px; font-size:1.1em;'>
                        <span style='font-weight:bold;'>Indoor:</span> {indoor_sensor_status_dynamic}
                    </p>
                    <p style='margin-bottom:0; font-size:1.1em;'>
                        <span style='font-weight:bold;'>Outdoor:</span> {outdoor_sensor_status_dynamic}
                    </p>
                </div>
                <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{LIGHTER_BLUE_COLOR}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                    <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                        <span style='font-size:1.5em; margin-right:10px;'>📋</span>Daily Tasks
                    </h4>
                    <p style='margin-bottom:0; font-size:1.1em;'><span style='font-weight:bold;'>{len(daily_tasks)}</span> tasks pending</p>
                </div>
                <div style='flex:1; min-width:280px; padding:15px; margin:0; background-color:{LIGHTER_BLUE_COLOR}; border-radius:10px; box-shadow: 0px 2px 5px rgba(0,0,0,0.1); text-align:center;'>
                    <h4 style='color:{DARKER_BLUE_COLOR}; margin-top:0; margin-bottom:10px; display:flex; align-items:center; justify-content:center;'>
                        <span style='font-size:1.5em; margin-right:10px;'>🏆</span>Optimization Race
                    </h4>
                    <p style='margin-bottom:0; font-size:1.1em;'>Leader: <span style='font-weight:bold;'>Charlie</span> with <span style='color:{ORANGE_COLOR}; font-weight:bold;'>1800</span> points</p>
                </div>
            </div>
        </div>
        """

    displayed_screen.children = [main_menu_container]
    display(displayed_screen)

def show_manager_screen() -> None:
    """Displays the manager screen."""
    global current_screen, displayed_screen
    current_screen = "manager"
    clear_output(wait=True)
    displayed_screen.children = [manager_container]
    display(displayed_screen)
    update_sensor_status()
    update_manager_alert()  # <-- Add this line

def show_query_screen() -> None:
    """Displays the query screen."""
    global current_screen, displayed_screen
    current_screen = "query"
    clear_output(wait=True)
    displayed_screen.children = [query_container]
    display(displayed_screen)

def show_statistics_screen() -> None:
    """Displays the statistics screen."""
    global current_screen, displayed_screen
    current_screen = "statistics"
    clear_output(wait=True)
    displayed_screen.children = [stats_container]
    display(displayed_screen)

    # Initialize graph selector options based on the default sensor
    if sensor_selector.value == "Indoor Sensor":
        graph_selector.options = ["Humidity", "Temperature", "Pressure", "Distance"]
    elif sensor_selector.value == "Outdoor Sensor":
        graph_selector.options = ["Humidity", "Temperature", "DLight"]

    # Trigger an initial update for both statistics and plot
    update_sensor_data_and_plot(None) # Call with None as change object is not relevant for initial load

def show_optimization_screen() -> None:
    """Displays the optimization screen."""
    global current_screen, displayed_screen
    current_screen = "optimization"
    clear_output(wait=True)
    displayed_screen.children = [optimization_container]
    display(displayed_screen)
    update_optimization_leaderboard()

def update_screen() -> None:
    """Updates the displayed screen based on the current screen variable."""
    if current_screen == "main_menu":
        show_main_menu()
    elif current_screen == "manager":
        show_manager_screen()
    elif current_screen == "query":
        show_query_screen()
    elif current_screen == "statistics":
        show_statistics_screen()
    elif current_screen == "optimization":
        show_optimization_screen()

<h2>Section 10: Event Handlers</h2>
<p>This section defines all event handler functions for user interactions, such as button clicks and dropdown changes. These handlers connect the UI widgets to their respective functionalities, enabling dynamic updates and navigation throughout the dashboard.</p>

In [ ]:
# --- Event Handlers ---
def handle_main_menu_navigation(button: widgets.Button) -> None:
    """Handles navigation from the main menu screen."""
    if button.description == "System Manager":
        show_manager_screen()
    elif button.description == "Sensor Statistics":
        show_statistics_screen()
    elif button.description == "Search Query":
        show_query_screen()
        query_input.value = ""  # Clear the text input box
        query_results.clear_output()  # Clear previous search results

    elif button.description == "Optimization Race":
        show_optimization_screen()

def handle_back_to_menu(button: widgets.Button) -> None:
    """Handles back to main menu navigation from any screen."""
    show_main_menu()

def handle_search(button: widgets.Button) -> None:
    """Handles the search query and displays results."""
    query = query_input.value
    results = perform_search(query)
    query_results.clear_output(wait=True)
    with query_results:
        display(widgets.HTML("<br>".join(results)))

def on_sensor_selector_change(change) -> None:
    """Updates graph options and then the plot based on the selected sensor."""
    selected_sensor = change['new']
    if selected_sensor == "Indoor Sensor":
        graph_selector.options = ["Humidity", "Temperature", "Pressure", "Distance"]
    elif selected_sensor == "Outdoor Sensor":
        graph_selector.options = ["Humidity", "Temperature", "DLight"]
    # Ensure the graph selector has a valid option selected after options change
    # IMPORTANT: Keep this check to prevent errors if the new sensor doesn't have the previously selected attribute
    if graph_selector.value not in graph_selector.options:
        graph_selector.value = graph_selector.options[0] if graph_selector.options else None

    # Trigger an update for both statistics and plot
    update_sensor_data_and_plot(None) # Call with None as change object is not relevant here

<h2>Section 11: Widget Event Assignments</h2>
<p>This section assigns the event handler functions to the corresponding widgets, such as buttons and dropdowns. By linking UI elements to their logic, this section ensures that user actions trigger the correct responses in the dashboard.</p>

In [ ]:
# --- Widget Event Assignments ---
manager_btn.on_click(handle_main_menu_navigation)
statistics_btn.on_click(handle_main_menu_navigation)
search_btn.on_click(handle_main_menu_navigation)
optimization_btn.on_click(handle_main_menu_navigation)

manager_back_btn.on_click(handle_back_to_menu)
stats_back_btn.on_click(handle_back_to_menu)
query_back_btn.on_click(handle_back_to_menu)
optimization_back_btn.on_click(handle_back_to_menu)

search_button.on_click(handle_search)

sensor_selector.observe(on_sensor_selector_change, names='value')
graph_selector.observe(update_sensor_data_and_plot, names='value')

<h2>Section 12: Initial Display and Startup</h2>
<p>This final section triggers the initial display of the main menu when the notebook is run. It ensures that users are greeted with the dashboard interface immediately, providing a seamless start to their interaction with the system.</p>

In [ ]:
# --- Initial Display ---
show_main_menu()

<h2>Section 13: MQTT and Data Simulation Utilities</h2>
<p>
This section contains utility code for MQTT integration, data simulation, and index creation. It includes commented-out code blocks for establishing MQTT connections, simulating sensor data uploads to Firebase, and building a searchable index from web content. These utilities are useful for testing, development, and extending the dashboard's capabilities.
</p>


In [ ]:

# !pip install paho-mqtt
# !pip install firebase
# import time
# import json
# import paho.mqtt.client as mqtt
# from datetime import datetime
# import pytz
# from firebase import firebase


# class MqttConnection:
#     _instance = None  # Singleton instance

#     @classmethod
#     def get_instance(cls, DBLink=None):
#         if cls._instance is None and DBLink is not None:
#             cls._instance = cls(DBLink)
#         return cls._instance

#     def __init__(self, DBLink):
#         if MqttConnection._instance is not None:
#             raise Exception("MqttConnection is a singleton.")

#         self.DBLink = DBLink
#         self.FBconn = firebase.FirebaseApplication(DBLink, None)
#         self.client = mqtt.Client(mqtt.CallbackAPIVersion.VERSION2)
#         self.connected = False

#     def insert_to_db(self, path, data):
#         try:
#             # Read existing data at this path
#             existing = self.FBconn.get('/', f'Data/{path}')
#             if existing is None:
#                 existing = {}

#             # Merge existing data with new data (new data overwrites existing keys)
#             merged_data = {**existing, **data}

#             self.FBconn.put('/', f'Data/{path}', merged_data)
#             print(f"Data saved to Firebase at path: Data/{path}")
#         except Exception as e:
#             print(f"Failed to insert data to Firebase: {e}")



#     def on_connect(self, client, userdata, flags, rc, properties=None):
#         if rc == 0:
#             self.connected = True
#             print("✅ Connected to MQTT Broker. Subscribing to topics...")
#             client.subscribe("braude/D106/indoor")
#             client.subscribe("braude/D106/outdoor")
#             print("✅ Successfully subscribed to topics.")
#         else:
#             print(f"❌ Failed to connect, return code {rc}")
#             self.connected = False

#     def on_disconnect(self, client, userdata, rc, properties=None, reason_string=None):
#         self.connected = False
#         if rc != 0:
#             for i in range(5):
#                 wait_time = (i + 1) * 2
#                 print(f"🔄 Disconnected. Reconnecting in {wait_time}s (attempt {i+1}/5)...")
#                 time.sleep(wait_time)
#                 try:
#                     client.reconnect()
#                     print("✅ Reconnection attempt made.")
#                     break
#                 except Exception as e:
#                     print(f"❌ Reconnection failed: {e}")

#     # Moved inside the class
#     def on_message(self, client, userdata, msg):
#         topic = msg.topic
#         payload = msg.payload.decode('utf-8')
#         try:
#             sensor_data = json.loads(payload)
#             timestamp = int(time.time())

#             local_tz = pytz.timezone("Israel")
#             sensor_data["Time"] = datetime.now(local_tz).strftime("%H:%M:%S")
#             sensor_data["Date"] = datetime.now(local_tz).strftime("%Y-%m-%d")
#             print(sensor_data)
#             if topic == "braude/D106/indoor":
#                 # Now self refers to the MqttConnection instance
#                 self.insert_to_db(f"indoor/{timestamp}", sensor_data)
#             elif topic == "braude/D106/outdoor":
#                 # Now self refers to the MqttConnection instance
#                 self.insert_to_db(f"outdoor/{timestamp}", sensor_data)
#         except json.JSONDecodeError as e:
#             print(f"❌ JSON decode error: {e}\nPayload: {payload}")

#     # Moved inside the class
#     def mqtt_handler(self):
#         self.client.on_connect = self.on_connect
#         self.client.on_disconnect = self.on_disconnect
#         self.client.on_message = self.on_message

#         try:
#             print("🔌 Attempting to connect to MQTT broker...")
#             conn_result = self.client.connect("test.mosquitto.org", 1883, keepalive=60)
#             print(f"Connection result code: {conn_result}")
#             self.client.loop_start()

#             timeout = time.time() + 10
#             while time.time() < timeout and not self.connected:
#                 time.sleep(0.2)

#             return self.connected

#         except Exception as e:
#             print(f"❌ Connection failed: {e}")
#             return False


# # --- Main execution ---
# if __name__ == "__main__":
#     DBLink = "https://cloudteamwolf-default-rtdb.europe-west1.firebasedatabase.app"

#     print("🚀 Initializing MQTT connection...")
#     global_mqtt_connection = MqttConnection.get_instance(DBLink)

#     if not global_mqtt_connection.connected:
#         success = global_mqtt_connection.mqtt_handler()
#         print(f"🔍 MQTT connection result: {'Connected' if success else 'Failed'}")

#     # Let it run for a while to receive messages
#     try:
#         time.sleep(30)
#     finally:
#         global_mqtt_connection.client.loop_stop()
#         print("🛑 MQTT loop stopped.")

<h2>Section 14: Index Creation Utilities</h2>
<p>
This section provides utility code for building a searchable index from web content. It includes functions for fetching web pages, processing and stemming words, filtering stopwords, and uploading the top terms to Firebase. These tools support advanced search and information retrieval features within the dashboard.
</p>

In [ ]:
# import time
# !pip install firebase
# from firebase import firebase
# from datetime import datetime
# from zoneinfo import ZoneInfo  # Python 3.9+
# import matplotlib.pyplot as plt
# import matplotlib.dates as mdates
# import random

# # Connect to Firebase
# FBconn = firebase.FirebaseApplication(
#     'https://cloudteamwolf-default-rtdb.europe-west1.firebasedatabase.app', None
# )

# local_tz = ZoneInfo("Asia/Jerusalem")

# def generate_logical_data_points(n=20):
#     data_points = []

#     base_humidity = 41.16
#     base_temperature = 23.2
#     base_pressure = 1
#     base_distance = 50

#     for i in range(n):
#         # Simulate small random fluctuations
#         humidity = base_humidity + random.uniform(-0.2, 0.2)
#         temperature = base_temperature + random.uniform(-0.05, 0.05)
#         pressure = base_pressure + random.uniform(-0.1, 0.1)
#         distance = base_distance + random.uniform(-50, 50)

#         point = {
#             "Humidity": round(humidity, 2),
#             "Temperature": round(temperature, 2),
#             "Pressure": round(pressure, 2),
#             "Distance": round(distance, 2)
#         }
#         data_points.append(point)

#     return data_points

# def add_data_points(data_points_raw, delay=1):
#     for point in data_points_raw:
#         now = datetime.now(local_tz)
#         timestamp = int(time.time())

#         # Separate date and time strings (no milliseconds)
#         date_str = now.strftime("%Y-%m-%d")
#         time_str = now.strftime("%H:%M:%S")

#         # Clamp temperature between 20 and 25
#         temp = max(23.0, min(25.0, point["Temperature"]))
#         humidity = point["Humidity"]
#         pressure = point["Pressure"]
#         distance = point["Distance"]

#         indoor_data = {
#             "Temperature": round(temp, 2),
#             "Humidity": round(humidity, 2),
#             "Pressure": round(pressure, 2),
#             "Distance": round(distance, 2),
#             "Date": date_str,
#             "Time": time_str
#         }

#         outdoor_temp = max(23.0, min(25.0, temp + random.uniform(-0.5, 0.5)))
#         outdoor_humidity = max(41.0, min(43.0, humidity + random.uniform(-0.2, 0.2)))
#         dlight = random.randint(600, 800)

#         outdoor_data = {
#             "Temperature": round(outdoor_temp, 2),
#             "Humidity": round(outdoor_humidity, 2),
#             "DLight": dlight,
#             "Date": date_str,
#             "Time": time_str
#         }

#         FBconn.put("Data/indoor", timestamp, indoor_data)
#         FBconn.put("Data/outdoor", timestamp, outdoor_data)

#         print(f"Uploaded indoor and outdoor data at timestamp: {timestamp}")

#         time.sleep(5)

# data_points = generate_logical_data_points(50)  # Generate 50 points for testing
# add_data_points(data_points, delay=5)  # 5 second delay between uploads

<h2>Section 15: Web Indexing and Search Utilities</h2>
<p>
This section provides code and utilities for web content indexing, including fetching, processing, and stemming words from multiple URLs, filtering stopwords, and uploading the top terms to Firebase. These tools enable advanced search and information retrieval features for the dashboard, supporting robust query functionality.
</p>

In [ ]:

# !pip install requests beautifulsoup4 nltk firebase

# # Import modules
# import requests
# from bs4 import BeautifulSoup
# import re
# from nltk.stem import PorterStemmer
# from nltk.corpus import stopwords
# import nltk
# from firebase import firebase

# # Download stopwords
# nltk.download('stopwords')
# english_stop_words = set(stopwords.words('english'))

# # Firebase setup
# firebase_url = "https://cloudteamwolf-default-rtdb.europe-west1.firebasedatabase.app"
# FBconn = firebase.FirebaseApplication(firebase_url, None)

# # Pages to index
# seed_urls = {
#     1: 'https://mqtt.org/',
#     2: 'https://mqtt.org/documentation',
#     3: 'https://mqtt.org/software',
#     4: 'https://mqtt.org/getting-started/',
#     5: 'https://mqtt.org/mqtt-specification/',
#     6: 'https://mqtt.org/use-cases/',
#     7: 'https://mqtt.org/faq/',
# }

# # Fetch page content
# def fetch_page_text(url):
#     try:
#         response = requests.get(url)
#         if response.status_code == 200:
#             soup = BeautifulSoup(response.text, 'html.parser')
#             return soup.get_text()
#     except Exception as e:
#         print(f"Failed to fetch {url}: {e}")
#     return ""
# !pip install requests beautifulsoup4 nltk firebase
# import requests
# from bs4 import BeautifulSoup
# import re
# from nltk.stem import PorterStemmer
# from nltk.corpus import stopwords
# import nltk
# from firebase import firebase
# nltk.download('stopwords')
# english_stop_words = set(stopwords.words('english'))
# firebase_url = "https://cloudteamwolf-default-rtdb.europe-west1.firebasedatabase.app"
# FBconn = firebase.FirebaseApplication(firebase_url, None)
# seed_urls = {
#     1: 'https://mqtt.org/',
#     2: 'https://mqtt.org/documentation',
#     3: 'https://mqtt.org/software',
#     4: 'https://mqtt.org/getting-started/',
#     5: 'https://mqtt.org/mqtt-specification/',
#     6: 'https://mqtt.org/use-cases/',
#     7: 'https://mqtt.org/faq/',
# }
# def fetch_page_text(url):
#     try:
#         response = requests.get(url)
#         if response.status_code == 200:
#             soup = BeautifulSoup(response.text, 'html.parser')
#             return soup.get_text()
#     except Exception as e:
#         print(f"Failed to fetch {url}: {e}")
#     return ""
# def sanitize_key(key: str) -> str:
#     return re.sub(r'[.#$]', '_', key)
# def build_index(urls):
#     index = {}
#     stemmer = PorterStemmer()
#     for _, url in urls.items():
#         print(f"Fetching and processing {url} ...")
#         text = fetch_page_text(url)
#         words = re.findall(r'\w+', text.lower())
#         for word in words:
#             if word in english_stop_words or len(word) <= 2:
#                 continue
#             stemmed = stemmer.stem(word)
#             if stemmed not in index:
#                 index[stemmed] = {
#                     "term": stemmed,
#                     "DocIDs": {},
#                     "count": 0
#                 }
#             index[stemmed]["DocIDs"][url] = index[stemmed]["DocIDs"].get(url, 0) + 1
#             index[stemmed]["count"] += 1
#     for data in index.values():
#         data["DocIDs"] = [
#             {"url": url, "count": count}
#             for url, count in sorted(data["DocIDs"].items())
#         ]
#     return index
# full_index = build_index(seed_urls)
# top_100_terms = sorted(full_index.values(), key=lambda x: x["count"], reverse=True)[:100]
# top_100_index = {
#     sanitize_key(entry["term"]): {
#         "term": entry["term"],
#         "count": entry["count"],
#         "DocIDs": entry["DocIDs"]
#     }
#     for entry in top_100_terms
# }
# print("\nUploading Top 100 words to Firebase...")
# FBconn.put('/dataWarehouse', 'Index', top_100_index)
# print("Upload complete.")
# print("\nTop 100 words uploaded:")
# for entry in top_100_terms:
#     print(f"{entry['term']:15} → {entry['count']:4} total occurrences in:")
#     for site in entry["DocIDs"]:
#         print(f"  {site['url']} → {site['count']} times")
# # Build full index using URLs as "DocIDs"
# def build_index(urls):
#     index = {}
#     stemmer = PorterStemmer()

#     for _, url in urls.items():
#         print(f"Fetching and processing {url} ...")
#         text = fetch_page_text(url)
#         words = re.findall(r'\w+', text.lower())

#         for word in words:
#             if word in english_stop_words or len(word) <= 2:
#                 continue
#             stemmed = stemmer.stem(word)
#             if stemmed not in index:
#                 index[stemmed] = {
#                     "term": stemmed,
#                     "DocIDs": {},
#                     "count": 0
#                 }
#             # Count occurrences per URL
#             index[stemmed]["DocIDs"][url] = index[stemmed]["DocIDs"].get(url, 0) + 1
#             index[stemmed]["count"] += 1

#     # Convert DocIDs dict to sorted list of URLs + counts
#     for data in index.values():
#         data["DocIDs"] = [
#             {"url": url, "count": count}
#             for url, count in sorted(data["DocIDs"].items())
#         ]

#     return index

# # Build and sort index
# full_index = build_index(seed_urls)

# # Get top 100 terms by frequency
# top_100_terms = sorted(full_index.values(), key=lambda x: x["count"], reverse=True)[:100]

# # Prepare upload dict with sanitized keys for Firebase compatibility
# top_100_index = {
#     sanitize_key(entry["term"]): {
#         "term": entry["term"],
#         "count": entry["count"],
#         "DocIDs": entry["DocIDs"]
#     }
#     for entry in top_100_terms
# }

# # Upload to Firebase under dataWarehouse/Index
# print("\nUploading Top 100 words to Firebase...")
# FBconn.put('/Data', 'Index', top_100_index)
# print("Upload complete.")

# # Display results
# print("\nTop 100 words uploaded:")
# for entry in top_100_terms:
#     print(f"{entry['term']:15} → {entry['count']:4} total occurrences in:")
#     for site in entry["DocIDs"]:
#         print(f"  {site['url']} → {site['count']} times")

# Gemini Chatbot Integration
This cell implements a chatbot using Google's Gemini LLM. It uses the `google.generativeai` library to interact with Gemini, and the API key is securely loaded from Google Colab secrets. The chatbot is wrapped in a function and a widget, so you can call and display it from any part of your notebook. You can further improve this by customizing the UI, adding context-awareness, or integrating it with your dashboard's widgets.

In [ ]:
# Gemini Chatbot Cell
# Install the Gemini library if needed
!pip install --quiet google-generativeai

import google.generativeai as genai
import os

# Load Gemini API key from Colab secrets (or environment variable for local dev)
try:
    from google.colab import userdata
    GEMINI_API_KEY = userdata.get('GEMINI_API_KEY')
except ImportError:
    GEMINI_API_KEY = os.environ.get('GEMINI_API_KEY')

if not GEMINI_API_KEY:
    raise ValueError("Gemini API key not found. Please set it in Colab secrets or as an environment variable.")

genai.configure(api_key=GEMINI_API_KEY)

# Create a Gemini chat function
chat_model = genai.GenerativeModel('gemini-1.5-flash')
chat_session = chat_model.start_chat()

def gemini_chatbot_widget():
    input_box = widgets.Textarea(
        placeholder='Ask Gemini anything...',
        description='You:',
        layout=widgets.Layout(width='100%', height='60px')
    )
    send_btn = widgets.Button(description='Send', button_style='info')
    output_box = widgets.Output(layout=widgets.Layout(border='1px solid #ccc', padding='10px', min_height='80px'))
    chat_history = []

    def on_send(_):
        user_msg = input_box.value.strip()
        if not user_msg:
            return
        input_box.value = ''
        with output_box:
            print(f'You: {user_msg}')
        chat_history.append({'role': 'user', 'parts': [user_msg]})
        try:
            response = chat_session.send_message(user_msg)
            bot_msg = response.text
        except Exception as e:
            bot_msg = f"[Error: {e}]"
        chat_history.append({'role': 'model', 'parts': [bot_msg]})
        with output_box:
            print(f'Gemini: {bot_msg}')

    send_btn.on_click(on_send)
    chat_ui = widgets.VBox([
        output_box,
        widgets.HBox([input_box, send_btn])
    ])
    return chat_ui